# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 11 2022 4:48PM,250633,16,EMERSA-8763536,"Emersa Waterbox, LLC",Released
1,Nov 11 2022 4:40PM,250631,10,0086165601,ISDIN Corporation,Released
2,Nov 11 2022 4:38PM,250630,16,TASA-50012903,TASA USA Inc.,Released
3,Nov 11 2022 4:37PM,250629,10,0086165611,ISDIN Corporation,Released
4,Nov 11 2022 4:37PM,250629,10,0086165610,ISDIN Corporation,Released
5,Nov 11 2022 4:32PM,250628,10,0086165608,ISDIN Corporation,Released
6,Nov 11 2022 4:32PM,250628,10,0086165609,ISDIN Corporation,Released
7,Nov 11 2022 4:29PM,250627,10,0086165602,ISDIN Corporation,Released
8,Nov 11 2022 4:29PM,250627,10,0086165603,ISDIN Corporation,Released
9,Nov 11 2022 4:29PM,250627,10,0086165598,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,250628,Released,2
20,250629,Released,2
21,250630,Released,1
22,250631,Released,1
23,250633,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250628,NaN,2.0
250629,NaN,2.0
250630,NaN,1.0
250631,NaN,1.0
250633,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250428,1.0,0.0
250554,0.0,1.0
250565,0.0,1.0
250571,0.0,1.0
250573,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250428,1,0
250554,0,1
250565,0,1
250571,0,1
250573,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250428,1,0
1,250554,0,1
2,250565,0,1
3,250571,0,1
4,250573,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250428,1,
1,250554,,1
2,250565,,1
3,250571,,1
4,250573,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 11 2022 4:48PM,250633,16,"Emersa Waterbox, LLC"
1,Nov 11 2022 4:40PM,250631,10,ISDIN Corporation
2,Nov 11 2022 4:38PM,250630,16,TASA USA Inc.
3,Nov 11 2022 4:37PM,250629,10,ISDIN Corporation
5,Nov 11 2022 4:32PM,250628,10,ISDIN Corporation
7,Nov 11 2022 4:29PM,250627,10,ISDIN Corporation
21,Nov 11 2022 1:16PM,250612,10,Eywa Pharma Inc.
30,Nov 11 2022 1:06PM,250611,21,"NBTY Global, Inc."
31,Nov 11 2022 12:19PM,250603,21,"NBTY Global, Inc."
32,Nov 11 2022 12:15PM,250602,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 11 2022 4:48PM,250633,16,"Emersa Waterbox, LLC",,1
1,Nov 11 2022 4:40PM,250631,10,ISDIN Corporation,,1
2,Nov 11 2022 4:38PM,250630,16,TASA USA Inc.,,1
3,Nov 11 2022 4:37PM,250629,10,ISDIN Corporation,,2
4,Nov 11 2022 4:32PM,250628,10,ISDIN Corporation,,2
5,Nov 11 2022 4:29PM,250627,10,ISDIN Corporation,,14
6,Nov 11 2022 1:16PM,250612,10,Eywa Pharma Inc.,5,4
7,Nov 11 2022 1:06PM,250611,21,"NBTY Global, Inc.",,1
8,Nov 11 2022 12:19PM,250603,21,"NBTY Global, Inc.",,1
9,Nov 11 2022 12:15PM,250602,19,"NAPP Technologies, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 11 2022 4:48PM,250633,16,"Emersa Waterbox, LLC",1,
1,Nov 11 2022 4:40PM,250631,10,ISDIN Corporation,1,
2,Nov 11 2022 4:38PM,250630,16,TASA USA Inc.,1,
3,Nov 11 2022 4:37PM,250629,10,ISDIN Corporation,2,
4,Nov 11 2022 4:32PM,250628,10,ISDIN Corporation,2,
5,Nov 11 2022 4:29PM,250627,10,ISDIN Corporation,14,
6,Nov 11 2022 1:16PM,250612,10,Eywa Pharma Inc.,4,5
7,Nov 11 2022 1:06PM,250611,21,"NBTY Global, Inc.",1,
8,Nov 11 2022 12:19PM,250603,21,"NBTY Global, Inc.",1,
9,Nov 11 2022 12:15PM,250602,19,"NAPP Technologies, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 11 2022 4:48PM,250633,16,"Emersa Waterbox, LLC",1,
1,Nov 11 2022 4:40PM,250631,10,ISDIN Corporation,1,
2,Nov 11 2022 4:38PM,250630,16,TASA USA Inc.,1,
3,Nov 11 2022 4:37PM,250629,10,ISDIN Corporation,2,
4,Nov 11 2022 4:32PM,250628,10,ISDIN Corporation,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 11 2022 4:48PM,250633,16,"Emersa Waterbox, LLC",1.0,NaN
1,Nov 11 2022 4:40PM,250631,10,ISDIN Corporation,1.0,NaN
2,Nov 11 2022 4:38PM,250630,16,TASA USA Inc.,1.0,NaN
3,Nov 11 2022 4:37PM,250629,10,ISDIN Corporation,2.0,NaN
4,Nov 11 2022 4:32PM,250628,10,ISDIN Corporation,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 11 2022 4:48PM,250633,16,"Emersa Waterbox, LLC",1.0,0.0
1,Nov 11 2022 4:40PM,250631,10,ISDIN Corporation,1.0,0.0
2,Nov 11 2022 4:38PM,250630,16,TASA USA Inc.,1.0,0.0
3,Nov 11 2022 4:37PM,250629,10,ISDIN Corporation,2.0,0.0
4,Nov 11 2022 4:32PM,250628,10,ISDIN Corporation,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2506025,28.0,6.0
15,250584,1.0,0.0
16,1002282,2.0,3.0
19,751775,3.0,0.0
21,751768,3.0,0.0
22,250601,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2506025,28.0,6.0
1,15,250584,1.0,0.0
2,16,1002282,2.0,3.0
3,19,751775,3.0,0.0
4,21,751768,3.0,0.0
5,22,250601,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,28.0,6.0
1,15,1.0,0.0
2,16,2.0,3.0
3,19,3.0,0.0
4,21,3.0,0.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,28.0
1,15,Released,1.0
2,16,Released,2.0
3,19,Released,3.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Executing,6.0,0.0,3.0,0.0,0.0,0.0
Released,28.0,1.0,2.0,3.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Executing,6.0,0.0,3.0,0.0,0.0,0.0
1,Released,28.0,1.0,2.0,3.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Executing,6.0,0.0,3.0,0.0,0.0,0.0
1,Released,28.0,1.0,2.0,3.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()